In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model

2025-12-19 10:58:13.587641: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766141894.000649      23 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766141894.102108      23 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766141895.221006      23 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766141895.221047      23 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766141895.221050      23 computation_placer.cc:177] computation placer alr

In [2]:
!pip install split-folders

In [3]:
import splitfolders

input_folder = "/kaggle/input/plantvillage-dataset/color"
output_folder = "/kaggle/working/PlantVillage_Split"

splitfolders.ratio(
    input_folder,
    output=output_folder,
    seed=42,
    ratio=(.8, .1, .1)   # train, val, test
)

Copying files: 54305 files [09:37, 94.00 files/s] 


In [4]:
import glob
import os

folders = glob.glob("/kaggle/working/PlantVillage_Split/train/*")
folder_names = sorted(os.path.basename(f) for f in folders)

folder_names



['Apple___Apple_scab',
 'Apple___Black_rot',
 'Apple___Cedar_apple_rust',
 'Apple___healthy',
 'Blueberry___healthy',
 'Cherry_(including_sour)___Powdery_mildew',
 'Cherry_(including_sour)___healthy',
 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
 'Corn_(maize)___Common_rust_',
 'Corn_(maize)___Northern_Leaf_Blight',
 'Corn_(maize)___healthy',
 'Grape___Black_rot',
 'Grape___Esca_(Black_Measles)',
 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
 'Grape___healthy',
 'Orange___Haunglongbing_(Citrus_greening)',
 'Peach___Bacterial_spot',
 'Peach___healthy',
 'Pepper,_bell___Bacterial_spot',
 'Pepper,_bell___healthy',
 'Potato___Early_blight',
 'Potato___Late_blight',
 'Potato___healthy',
 'Raspberry___healthy',
 'Soybean___healthy',
 'Squash___Powdery_mildew',
 'Strawberry___Leaf_scorch',
 'Strawberry___healthy',
 'Tomato___Bacterial_spot',
 'Tomato___Early_blight',
 'Tomato___Late_blight',
 'Tomato___Leaf_Mold',
 'Tomato___Septoria_leaf_spot',
 'Tomato___Spider_mites Two-spotted_

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [6]:
img=256
b_s=32
train_datagen= ImageDataGenerator(
    rescale=1/255.,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
)

In [7]:
valid_test_datagen=ImageDataGenerator(rescale=1/255.)

In [8]:
train_dir="/kaggle/working/PlantVillage_Split/train"
valid_dir="/kaggle/working/PlantVillage_Split/val"
test_dir="/kaggle/working/PlantVillage_Split/test"

In [9]:
train=train_datagen.flow_from_directory(
    train_dir,
    target_size=(img,img),
    batch_size=b_s,
    class_mode="categorical",
    shuffle=True,
    seed=123
)

Found 43429 images belonging to 38 classes.


In [10]:
train[0]

(array([[[[0.41138694, 0.4035438 , 0.44668105],
          [0.40997183, 0.4021287 , 0.44526595],
          [0.40794098, 0.40009785, 0.44323513],
          ...,
          [0.29134706, 0.27958235, 0.3227196 ],
          [0.30681852, 0.2950538 , 0.33819106],
          [0.31122607, 0.29946136, 0.34259862]],
 
         [[0.4214197 , 0.41357657, 0.45671383],
          [0.41838014, 0.410537  , 0.45367426],
          [0.4104819 , 0.40263876, 0.44577605],
          ...,
          [0.29577735, 0.28401265, 0.3271499 ],
          [0.30871722, 0.29695252, 0.34008977],
          [0.30932736, 0.29756266, 0.3406999 ]],
 
         [[0.43471852, 0.42687538, 0.47001263],
          [0.43646643, 0.4286233 , 0.47176057],
          [0.43432024, 0.4264771 , 0.46961436],
          ...,
          [0.3002076 , 0.2884429 , 0.3315802 ],
          [0.3106159 , 0.2988512 , 0.34198844],
          [0.30742866, 0.29566395, 0.3388012 ]],
 
         ...,
 
         [[0.50749683, 0.48396745, 0.5231831 ],
          [0.47614

In [11]:
valid=valid_test_datagen.flow_from_directory(
    valid_dir,
    target_size=(img,img),
    batch_size=b_s,
    class_mode="categorical",
    shuffle=False
)

Found 5417 images belonging to 38 classes.


In [12]:
test=valid_test_datagen.flow_from_directory(
    test_dir,
    target_size=(img,img),
    batch_size=b_s,
    class_mode="categorical",
    shuffle=False
)

Found 5459 images belonging to 38 classes.


In [13]:
classes=train.num_classes
classes

38

In [14]:
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout

In [15]:
from tensorflow import keras

In [16]:
model = keras.models.Sequential([
    keras.layers.Conv2D(filters=32, kernel_size=3, activation="relu", input_shape=[256, 256, 3]),
    keras.layers.MaxPool2D(pool_size=(2, 2)),
    
    keras.layers.Conv2D(filters=64, kernel_size=3, activation="relu"),
    keras.layers.MaxPool2D(pool_size=(2, 2)),
    
    keras.layers.Conv2D(filters=128, kernel_size=3, activation="relu"),
    keras.layers.MaxPool2D(pool_size=(2, 2)),
    
    # NEW: Added another layer to capture more complex plant patterns
    keras.layers.Conv2D(filters=256, kernel_size=3, activation="relu"),
    keras.layers.GlobalAveragePooling2D(), # Better than Flatten for large datasets
    
    keras.layers.Dense(units=256, activation="relu"),
    keras.layers.Dropout(0.4), # Increased slightly for better regularization
    keras.layers.Dense(units=38, activation="softmax"),
])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1766142504.627716      23 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1766142504.631610      23 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


In [17]:
from tensorflow.keras.optimizers import Adam

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)


In [18]:
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau, ModelCheckpoint

In [19]:
cb = [
    EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=3),
    ModelCheckpoint("best_model_plant_50000.keras",monitor="val_loss",save_best_only=True)
]

In [20]:
history = model.fit(
    train,
    epochs=200,
    validation_data=valid,
    callbacks=cb
)


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/200


I0000 00:00:1766142511.541397      96 service.cc:152] XLA service 0x7a9e6c10f430 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1766142511.541434      96 service.cc:160]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1766142511.541439      96 service.cc:160]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1766142512.188277      96 cuda_dnn.cc:529] Loaded cuDNN version 91002
2025-12-19 11:08:37.195051: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-12-19 11:08:37.488442: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


   2/1358 ━━━━━━━━━━━━━━━━━━━━ 2:00 89ms/step - accuracy: 0.0078 - loss: 3.6300      

I0000 00:00:1766142521.133297      96 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1199/1358 ━━━━━━━━━━━━━━━━━━━━ 1:03 402ms/step - accuracy: 0.2407 - loss: 2.8325

2025-12-19 11:16:44.765344: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-12-19 11:16:45.044039: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


1358/1358 ━━━━━━━━━━━━━━━━━━━━ 570s 412ms/step - accuracy: 0.2530 - loss: 2.7745 - val_accuracy: 0.5208 - val_loss: 1.6559 - learning_rate: 0.0010
Epoch 2/200
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 546s 402ms/step - accuracy: 0.5921 - loss: 1.3587 - val_accuracy: 0.7236 - val_loss: 0.8648 - learning_rate: 0.0010
Epoch 3/200
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 552s 406ms/step - accuracy: 0.7145 - loss: 0.9316 - val_accuracy: 0.8204 - val_loss: 0.5658 - learning_rate: 0.0010
Epoch 4/200
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 573s 422ms/step - accuracy: 0.7733 - loss: 0.7282 - val_accuracy: 0.8357 - val_loss: 0.5210 - learning_rate: 0.0010
Epoch 5/200
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 594s 438ms/step - accuracy: 0.8142 - loss: 0.5846 - val_accuracy: 0.8558 - val_loss: 0.4410 - learning_rate: 0.0010
Epoch 6/200
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 588s 433ms/step - accuracy: 0.8406 - loss: 0.5058 - val_accuracy: 0.8691 - val_loss: 0.4083 - learning_rate: 0.0010
Epoch 7/200
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 589s 433ms/step

In [21]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

y_pred_probs = model.predict(test)
y_pred = np.argmax(y_pred_probs, axis=1)

y_true = test.classes

class_names = list(test.class_indices.keys())

cm = confusion_matrix(y_true, y_pred)
print(cm)

print(classification_report(y_true, y_pred, target_names=class_names))


171/171 ━━━━━━━━━━━━━━━━━━━━ 9s 51ms/step
[[ 59   1   0 ...   0   0   0]
 [  0  63   0 ...   0   0   0]
 [  0   0  27 ...   0   0   0]
 ...
 [  0   0   0 ... 533   0   0]
 [  0   0   0 ...   0  38   0]
 [  0   0   0 ...   0   0 160]]
                                                    precision    recall  f1-score   support

                                Apple___Apple_scab       1.00      0.94      0.97        63
                                 Apple___Black_rot       0.98      1.00      0.99        63
                          Apple___Cedar_apple_rust       1.00      0.96      0.98        28
                                   Apple___healthy       0.98      1.00      0.99       165
                               Blueberry___healthy       1.00      1.00      1.00       151
          Cherry_(including_sour)___Powdery_mildew       1.00      0.98      0.99       106
                 Cherry_(including_sour)___healthy       1.00      0.97      0.98        86
Corn_(maize)___Cercospora_lea

In [22]:
model.evaluate(test)

171/171 ━━━━━━━━━━━━━━━━━━━━ 7s 43ms/step - accuracy: 0.9863 - loss: 0.0364


[0.033673811703920364, 0.988825798034668]